In [281]:
import pandas as pd
import time
import subprocess
model_name='/datadrive/model_all'
files_path='/datadrive/libffm_data/'
files_name='libffm_w_all_features'
test_file_index=1
train_file=files_path+files_name+'0'
test_file=files_path+files_name+str(test_file_index)
output_file='output'+str(test_file_index)+'_'+str(output_index)
output_index+=1

In [167]:
output_index=1

In [133]:
#split the validation set from training...

df_train = pd.read_csv("clicks_train.csv") 
df_train.head()

df_train_1=df_train[df_train.display_id % 10 ==test_file_index]
df_train_1=df_train_1.reset_index()
print 1

In [ ]:
#run the file
t=time.time()
ouput=subprocess.check_output(['sudo','pypy','libffm_v2.py',files_name])
# args: file name, 
print time.time()-t

In [245]:
train_file

'/datadrive/libffm_data/libffm_w_meta0'

In [ ]:
#train model

model_param_tags={'regularization':'-l','model_depth':'-k','epcohs':'-t','learn_rate':'-r','cores':'-s',\
                 'validation_file':'-p'}
model_params={'regularization':'0.002','model_depth':'16','epcohs':'13','learn_rate':'0.1','cores':'12'}

t=time.time()
#train
results=subprocess.check_output(['sudo','../../libffm/ffm-train','-l','0.002','-k','12','-t','30','-r','0.1','-s','12'\
                        ,'-p',test_file,'--auto-stop',train_file,model_name])


print 'model trained in',t-time.time()
print results
#sudo ../ libffm_format_feats_0 /datadrive/model_3 /datadrive/output
#../../../libffm/ffm-train -l 0.002 -k 16 -t 13 -r 0.1 -s 12 -p libffm_format_feats_0 libffm_format_feats_1 /datadrive/model

#size 2^23=8M*k=16*features=6
#2^23*2^4*6

In [182]:
model_name

'/datadrive/model9.1'

In [ ]:
#run the file for test
'''
t=time.time()
ouput=subprocess.check_output(['sudo','pypy','libffm_v2.py',files_path+files_name])
print time.time()-t

In [ ]:
print 1

In [249]:

#predict
t=time.time()
results=subprocess.check_output(['sudo','../../libffm/ffm-predict',test_file\
                                 ,model_name,'/datadrive/'+output_file])
print 'model predicted in',time.time()-t
print results

model predicted in 136.132448912
logloss = 0.42866



In [122]:
#for submission
df_train_1=pd.read_csv('clicks_test.csv')

In [ ]:
#read output

#ndont forget to reset index if necesary!


output_list=pd.read_csv('/datadrive/'+output_file,header=None)

print len(output_list),len(df_train_1)

#test_set=df_train_1[['display_id','ad_id']]# for submission
test_set=df_train_1[['display_id','ad_id','clicked']]
test_set['prediction']=output_list[0]
#join to test

# sort test set
#test_set=test_set.sort(['display_id','prediction'],ascending=[True,False])
test_set['prediction'].fillna(0)
test_set=test_set.rename(columns = {'prediction':'likelihood'})
#test_set.head()

In [30]:
#test_set['likelihood']=output_list
test_set.tail(2) #verify its not Nan :{}

,display_id,ad_id,clicked,likelihood
8714870,16874591,397506,0,0.454487
8714871,16874591,483448,0,0.525756


In [276]:
import util
#calc mapk on validation set
util.fast_mapk(test_set) #columns: display id, likelihood,clicked
print output_file
#0.483445906214
#0.576126738888
# 2feats 0.604136968694  ^24
# 3 feats logloss 0.45885 ^23 map ?
# 4 feats 0.44571 0.634
# 6 feats 0.43375,train_time=7min,predict_time=5min map=0.6534
#6,full data, 0.40, 15min train,
# 6 feats ,train_time=3min,predict_time=1.5 min #same as above with p=22, factors=8 map:0.65237
# 7 feats 0.43481,train_time=7min,predict_time=1.5 min #same as above with p=22, factors=8 map:6524
# 6 feats+leak 0.42786 0.66262
# 8 feats, leak 0.43129 0.6619
# 7 feats added all categories to dest doc - 0.4277 - 0.6627 - slight improvment
#10.1: added all feats from promoted content -0.43757 - 650 - output1_2
#removed the above, added source_doc cat 0.42805, 0.6625 - no improve output1_3
#added power 22 entity - 0.42906 - 0.6614 - no improve - output1_4
#raised power to 23. 0.42869 train>10min 0.6618 slight... -
#reduce power to 21 0.42889 -  0.6614 output1_5
#p=21, source_cat removed 0.42975 - 0.6609 turns out it does help 'output1_6'
#meta added 0.42866 0.6623 'output1_7'
# 1&7=0.663

0.0636279582977
0.407561063766
0.614371061325
0.00341701507568
0.00769591331482
MAP: 0.662206862465
output1_7


In [280]:
#read output

#ndont forget to reset index if necesary!

for i in range(1,7):
    output_list1=pd.read_csv('/datadrive/output1_'+str(i),header=None)
    for j in range(i,7):
        
        output_list2=pd.read_csv('/datadrive/output1_'+str(j),header=None)
        output_list=(output_list1+output_list2)/2

        #print len(output_list),len(df_train_1)

        #test_set=df_train_1[['display_id','ad_id']]# for submission
        test_set=df_train_1[['display_id','ad_id','clicked']]
        test_set['prediction']=output_list[0]
        #join to test

        # sort test set
        #test_set=test_set.sort(['display_id','prediction'],ascending=[True,False])
        test_set['prediction'].fillna(0)
        test_set=test_set.rename(columns = {'prediction':'likelihood'})
        #test_set.head()
        i,j,util.fast_mapk(test_set)

0.0707819461823
0.415048122406
0.632957935333
0.00375890731812
0.00748991966248
MAP: 0.662672321973
0.0698289871216
0.416141986847
0.627278089523
0.0033860206604
0.00854706764221
MAP: 0.662745454292
0.0801260471344
0.468549013138
0.700538873672
0.00355911254883
0.010064125061
MAP: 0.662327238896
0.0712351799011
0.422986030579
0.629547834396
0.00348997116089
0.00846600532532
MAP: 0.662437840203
0.0664689540863
0.410794019699
0.630261898041
0.00347518920898
0.00841903686523
MAP: 0.662336224668
0.0685999393463
0.416464090347
0.631026029587
0.00359511375427
0.00838398933411
MAP: 0.662210173473
0.0707120895386
0.414577007294
0.634953975677
0.00358009338379
0.00756192207336
MAP: 0.662530762249
0.0725491046906
0.419175863266
0.629643917084
0.00358510017395
0.00872993469238
MAP: 0.662144135651
0.0718870162964
0.422389984131
0.607280015945
0.00339698791504
0.0080988407135
MAP: 0.662363483547
0.0774989128113
0.448140144348
0.706998825073
0.00399088859558
0.00956010818481
MAP: 0.662181601325
0.05

In [253]:
test_set['new']=(test_set['likelihood']+test_set2['likelihood'])/2

In [264]:
#average outputs
output_list=(output_list+output_list2)/2

In [265]:
output_list

,0
0,0.160994
1,0.185767
2,0.343085
3,0.049693
4,0.185279
5,0.254314
6,0.277279
7,0.121072
8,0.153444
9,0.265559


In [126]:
#make a submission
first=1
f = open('submission','w')
f.write('display_id,ad_id')
for item in test_set.iterrows():
    if first or item[1]['display_id']<>pred_dis_id:
        #print item
        f.write('\n')
        f.write(str(int(item[1]['display_id']))+',')
    first=0
    pred_dis_id=item[1]['display_id']    
    f.write(str(int(item[1]['ad_id']))+' ') 
f.close()

In [188]:
#item[1]
import pandas as pd
